In [12]:
!pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
import torch
print(torch.__version__)  # This should output the PyTorch version
print(torch.cuda.is_available())  # This should return True if CUDA is enabled


2.0.1+cu117
True


In [14]:
!pip install --upgrade transformers


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [15]:
from transformers import pipeline
import json

# Initialize the Korean model for text generation
model = pipeline('text-generation', model='wolf010/4TH_fine_tuned_Llama-3.2-3B-Instruct', device=0)

# Initialize conversation history, summary history, and order confirmation flag
conversation_history = []  # Stores each customer input as a string entry in the format: "Customer's X Input: [input]"
summary_history = []       # Stores each cumulative summary of orders
order_confirmed = False    # Tracks if the order is finalized

# Function to generate prompt based on conversation history and new input
def generate_prompt(conversation_history, user_input):
    # Format the conversation history and cumulative summary as a single string
    if conversation_history:
        formatted_history = " ".join(conversation_history)
    else:
        formatted_history = "none"

    base_prompt = f"""
    You are operating a virtual coffee kiosk that receives speech-to-text (STT) inputs from customers placing coffee orders. Your role is to understand and process these inputs, respond naturally in Korean, and generate a structured JSON file with the correct details for backend processing.
    if **Current Conversation History**:
    {formatted_history}
    generate the appropriate natural language response and JSON output
    """
    return base_prompt.strip()

# Function to get response from the Korean model pipeline
def get_response(prompt):
    response = model(prompt, max_new_tokens=150, num_return_sequences=1, temperature=0.1, top_p=0.9, truncation=True)
    return response[0]['generated_text']

# Function to parse response into natural language and JSON output
def parse_response(response):
    try:
        natural_language_response, json_output = response.split("JSON Output:")
        json_data = json.loads(json_output.strip())
        return natural_language_response.strip(), json_data
    except ValueError as e:
        print("Error parsing response:", e)
        return response, None

# Main interaction loop
print("Welcome to the virtual coffee kiosk! What would you like to order?")
input_counter = 1

while not order_confirmed:
    # Take user input
    user_input = input("Customer: ")

    # Check if customer confirms the order
    if "주문 완료할게" in user_input:
        order_confirmed = True
        print("Kiosk: 주문이 완료되었습니다. 결제는 카드리더기를 사용해주세요. 감사합니다.")
        conversation_history.clear()
        summary_history.clear()
        continue

    # Add the latest user input to the conversation history
    conversation_history.append(f"Customer's {input_counter} Input: {user_input}")
    input_counter += 1

    # Generate the prompt based on the conversation history and new user input
    prompt = generate_prompt(conversation_history, user_input)

    # Get the model's response
    response = get_response(prompt)

    # Parse the model's response
    natural_language_response, json_output = parse_response(response)

    # Save natural language order summary to `summary_history`
    if natural_language_response:
        summary_line = natural_language_response.split("\n", 1)[1]  # Extracts summary part
        summary_history.append(summary_line)

    # Print the final natural language response for the customer
    #print("Kiosk:", natural_language_response)
    # Print only the Natural Language Response and JSON Output in the required format
    print(response) # Pretty-print JSON output

    # Special handling for order summary requests
    if "내가 지금까지 뭘 주문했지" in user_input:
        # Provide cumulative order summary as a natural response without JSON output
        print("Kiosk:", "지금까지의 주문내역입니다:\n" + "\n".join(summary_history))

    # Handle order cancellation requests
    if "주문 취소" in user_input:
        # Clear conversation history and summary history for a reset
        print("Kiosk: 주문이 취소되었습니다. 새 주문을 시작해 주세요.")
        conversation_history.clear()
        summary_history.clear()
        input_counter = 1

# End of the session
print("Thank you for using the coffee kiosk!")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
import torch

# Prints the current GPU memory usage
print(f"Allocated Memory: {torch.cuda.memory_allocated() / 1024 ** 2} MB")
print(f"Cached Memory: {torch.cuda.memory_reserved() / 1024 ** 2} MB")


In [ ]:
%whos


In [ ]:
del model  # Delete the model from memory
torch.cuda.empty_cache()  # Clear the GPU memory cache
